In [1]:
import pandas  as pd 
import numpy as np
import tensorflow as tf
import cv2

In [2]:
train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                                                                zoom_range=0.2,
                                                                shear_range=0.2,
                                                                horizontal_flip=True,
                                                                rescale=1/255)

In [3]:
training_dataset = train_data_gen.flow_from_directory(r'C:\Users\HP SSD\Deep Learning\Face Detection\train',
                     target_size=(150,150),
                     class_mode='binary',
                     batch_size=16)

Found 1313 images belonging to 2 classes.


In [4]:
testing_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

In [5]:
testing_dataset = testing_data_gen.flow_from_directory(r'C:\Users\HP SSD\Deep Learning\Face Detection\test',
                                                   target_size=(150,150),
                                                   class_mode='binary',
                                                   batch_size=16)

Found 194 images belonging to 2 classes.


In [6]:
face_model = tf.keras.models.Sequential()


face_model.add(tf.keras.layers.Conv2D(40,(3,3),activation='relu',input_shape=(150,150,3)))
face_model.add(tf.keras.layers.Conv2D(80,(3,3),activation='relu'))
face_model.add(tf.keras.layers.MaxPool2D(2,2))

face_model.add(tf.keras.layers.Conv2D(120,(3,3),activation='relu'))
face_model.add(tf.keras.layers.MaxPool2D(2,2))

face_model.add(tf.keras.layers.Flatten())


face_model.add(tf.keras.layers.Dense(units=50,activation='relu'))
face_model.add(tf.keras.layers.Dense(units=50,activation='relu'))
face_model.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

face_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [7]:
face_model.fit(training_dataset,validation_data=testing_dataset,epochs=3)

Epoch 1/3
83/83 [==============================] - 253s 3s/step - loss: 0.5401 - accuracy: 0.7609 - val_loss: 0.2506 - val_accuracy: 0.9021
Epoch 2/3
83/83 [==============================] - 236s 3s/step - loss: 0.1803 - accuracy: 0.9307 - val_loss: 0.2206 - val_accuracy: 0.9124
Epoch 3/3
83/83 [==============================] - 229s 3s/step - loss: 0.1085 - accuracy: 0.9566 - val_loss: 0.0907 - val_accuracy: 0.9691


In [9]:
face_model.save('face_model.keras')

In [10]:
model = tf.keras.models.load_model('face_model.keras')

In [11]:
cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [12]:
cap=cv2.VideoCapture(0)

while True:
       _,frame = cap.read()
       faces = cascade.detectMultiScale(frame,scaleFactor=1.1)
       for x,y,w,h in faces:
            face=frame[y:y+h,x:x+w]
            cv2.imwrite("face.jpg",face)
            face = tf.keras.preprocessing.image.load_img("face.jpg",target_size=(150,150,3))#Convert to  numpy
            face = tf.keras.preprocessing.image.img_to_array(face)
            face = np.expand_dims(face,axis=0) #Convert into 4d
            pred = model.predict(face)
            if pred>0.6:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)
                cv2.putText(frame," NO Mask", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            else:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
                cv2.putText(frame,"Mask", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
       cv2.imshow("window",frame)
       if cv2.waitKey(1)==ord('q'):
           break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 73ms/step
